## Inferences on Decoding parameters in steering GPT2 sequences from scratch

HF Library tools used: https://huggingface.co/docs/transformers/en/internal/generation_utils

## Adding mask ids to the Tokenizer 

In [320]:
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn 
from transformers import GPT2Tokenizer, GPT2LMHeadModel, LogitsProcessor, LogitsProcessorList, set_seed

DIAL_TOKEN = ["<|user|>","<|agent|>"]
# INTENT_TOKEN = ["<ANSWER>", "<QUESTION>", "<COMMAND>", "<EXPRESSION>"]
# EMOTE_TOKEN = ["UNKNOWN","<NEUTRAL>", '<EMOTION:POS>', '<EMOTION:NEG>']

set_seed(42)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
# pad_token_id == eos_token_id == bos_token_id == <|endoftext|> 
tokenizer.pad_token = tokenizer.eos_token
# tokenizer.add_tokens(DIAL_TOKEN, special_tokens=False)
# model.resize_token_embeddings(len(tokenizer))
tokenizer.add_special_tokens

<bound method SpecialTokensMixin.add_special_tokens of GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}>

#### Ensuring the configuration of the tokens are as expected (sometimes you can be calling the wrong model or that the new update may have removed / altered some layers / parameters)

In [30]:
tokenizer.added_tokens_decoder

{50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True)}

In [31]:
tokenizer.added_tokens_encoder

{'<|endoftext|>': 50256}

#### Default generation from gpt2

In [63]:
# define inputs
user_input = "User: I love you, do you love me?.\nAgent:"

# preprocessing the inputs 
inputs = tokenizer(user_input, add_special_tokens=False, return_tensors="pt")
input_length = inputs.input_ids.size(1)
generation_output = model.generate(
    **inputs, 
    # max_length=input_length+50, 
    max_new_tokens=50,
    num_return_sequences=1, 
    no_repeat_ngram_size=4, 
    temperature=1.0, 
    do_sample=True, 
    return_dict_in_generate=True, 
    #logits_processor=custom_logits, 
    output_scores=True
)

# cleans text
print(tokenizer.decode(generation_output.sequences.squeeze(0), skip_special_tokens=False))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User: I love you, do you love me?.
Agent: Ohhh. Not that I know. You need me to talk to a bunch of women. I love you.
Linda shakes her head. "No," she says before she can finish.
Bryan grins. "And you're


#### Summary table from the generation config's documentation (Decoder parameters accepted for this library)

| **Arg**                     | **Usage**                                                                                                           | **Type**                           |
|-----------------------------|---------------------------------------------------------------------------------------------------------------------|-----------------------------------|
| `max_length`               | Maximum length of generated tokens (prompt + `max_new_tokens`). Overridden by `max_new_tokens` if set.              | `int` (default: 20)              |
| `max_new_tokens`           | Maximum number of tokens to generate, ignoring the prompt length.                                                   | `int`                            |
| `min_length`               | Minimum length of the sequence to be generated (prompt + `min_new_tokens`). Overridden by `min_new_tokens`.         | `int` (default: 0)               |
| `min_new_tokens`           | Minimum number of tokens to generate, ignoring the prompt length.                                                   | `int`                            |
| `early_stopping`           | Stops beam-based methods early: `True` (stops when `num_beams` candidates are complete), `False` (heuristic), `"never"`. | `bool` or `str` (default: `False`) |
| `max_time`                 | Maximum runtime for generation in seconds. Generation finishes the current pass even if time exceeds.               | `float`                          |
| `stop_strings`             | String(s) that terminate generation if output by the model.                                                         | `str` or `List[str]`             |
| `do_sample`                | Enables multinomial sampling; otherwise uses greedy decoding.                                                       | `bool` (default: `False`)        |
| `num_beams`                | Number of beams for beam search. If `1`, no beam search is performed.                                                | `int` (default: 1)               |
| `num_beam_groups`          | Divides `num_beams` into groups for diverse beam search.                                                            | `int` (default: 1)               |
| `penalty_alpha`            | Balances model confidence and degeneration penalty in contrastive search.                                           | `float`                          |
| `use_cache`                | Whether to use the model's past key/value cache for faster decoding.                                                | `bool` (default: `True`)         |
| `temperature`              | Modulates token probabilities; higher values increase randomness.                                                  | `float` (default: 1.0)           |
| `top_k`                    | Retains only the top-k probability tokens.                                                                          | `int` (default: 50)              |
| `top_p`                    | Retains the smallest set of tokens with cumulative probabilities ≥ `top_p`.                                         | `float` (default: 1.0)           |
| `min_p`                    | Minimum token probability scaled by the most likely token.                                                         | `float`                          |
| `typical_p`                | Retains locally typical tokens with probabilities adding up to `typical_p`.                                         | `float` (default: 1.0)           |
| `epsilon_cutoff`           | Filters tokens with conditional probabilities below `epsilon_cutoff`.                                               | `float` (default: 0.0)           |
| `eta_cutoff`               | Hybrid of typical and epsilon sampling with entropy scaling.                                                        | `float` (default: 0.0)           |
| `diversity_penalty`        | Penalizes beams that generate tokens appearing in other groups during group beam search.                            | `float` (default: 0.0)           |
| `repetition_penalty`       | Penalizes repeated sequences. Higher values reduce repetition.                                                      | `float` (default: 1.0)           |
| `length_penalty`           | Exponential penalty for sequence length in beam-based methods. Longer sequences favored if > 1.0.                   | `float` (default: 1.0)           |
| `no_repeat_ngram_size`     | Ensures n-grams of this size do not repeat during generation.                                                       | `int` (default: 0)               |
| `bad_words_ids`            | Token IDs disallowed in generated sequences.                                                                        | `List[List[int]]`                |
| `forced_bos_token_id`      | Forces the first generated token to be a specific token.                                                            | `int`                            |
| `forced_eos_token_id`      | Forces the last generated token(s) to be specific tokens.                                                           | `int` or `List[int]`             |
| `remove_invalid_values`    | Removes `NaN` or `Inf` values from logits to avoid crashes.                                                         | `bool`                           |
| `num_return_sequences`     | Number of independently generated sequences per input.                                                              | `int` (default: 1)               |
| `output_attentions`        | Returns attention scores if set to `True`.                                                                          | `bool` (default: `False`)        |
| `output_hidden_states`     | Returns hidden states if set to `True`.                                                                             | `bool` (default: `False`)        |
| `output_scores`            | Returns prediction scores if set to `True`.                                                                         | `bool` (default: `False`)        |
| `pad_token_id`             | Token ID for padding.                                                                                               | `int`                            |
| `bos_token_id`             | Token ID for beginning of sequence.                                                                                 | `int`                            |
| `eos_token_id`             | Token ID for end of sequence.                                                                                       | `int` or `List[int]`             |

In [323]:
history = []
user_input = "User: I love you. Do you love me?\nAgent:"

In [ ]:
class Plankton(LogitsProcessor):
    """
    Logit Scores Custom Altering first attempt. 
    NOTE: This actually iterates token by token NOT sequence by sequence
    """ 
    
    def __init__(self, input_size: int, boost_tokens, anchor_tokens, boost_scale: int, anchor_scale: int = None):
        super().__init__()
        self.input_size = input_size
        self.anchor_tokens = [i for i in anchor_tokens if i != tokenizer.eos_token_id]
        self.boost_tokens = [i for i in boost_tokens if i != tokenizer.eos_token_id]
        self.boost_scale = boost_scale 
        self.anchor_scale = anchor_scale if anchor_scale is not None else self.boost_scale
        self.counter = 0 
        
    def __call__(self, input_ids, logits):
        """This is called very token logits (shape: [1, 50259]). It prints out the scaled logits and decoded token's ids."""

        # logits shape is [1, 50259] ~ [1, vocab_dim]
        logits[:, self.anchor_tokens] -= self.anchor_scale # logits shape being scaled [1, 4] ~ [1, anchor_tokens]
        logits[:, self.boost_tokens] += self.boost_scale # alike anchor, [1, boost_tokens]

        print(f">>>>>>>>>>>>>>>>>>> Function called {self.counter+1}: {tokenizer.decode(input_ids[:, self.input_size:].squeeze(0), skip_special_tokens=False)} \nAnchor Logits: {logits[:, self.anchor_tokens].cpu().numpy()}\nBoost Logits: {logits[:, self.boost_tokens].cpu().numpy()}")
        
        self.counter += 1
        
        return logits

In [338]:
# trying out 1 token
custom_logit = Plankton(
    input_size=input_length,
    anchor_tokens=tokenizer.convert_tokens_to_ids(["hate"]),
    boost_tokens=tokenizer.convert_tokens_to_ids(["love", "adore"]),
    boost_scale=10
)

custom_logits = LogitsProcessorList([custom_logit])
inputs = tokenizer(user_input, add_special_tokens=False, return_tensors="pt")
input_length = inputs.input_ids.size(1)
# fixed variables - remains unchanged throughout
generation_output = model.generate(
    **inputs, 
    max_length=input_length+50, 
    num_return_sequences=1, 
    top_k=50,
    top_p=0.7,
    temperature=0.95, 
    stop_strings=["Agent:", "User:", "\n"],
    do_sample=True, 
    return_dict_in_generate=True, 
    tokenizer=tokenizer,
    logits_processor=custom_logits, 
    output_scores=True,
    use_cache=False,
)
output_tokens = tokenizer.decode(generation_output.sequences[:, input_length:].squeeze(0), skip_special_tokens=False)
history.append({
    "inputs": user_input, 
    "outputs": output_tokens,
    **custom_logit.__dict__
})

print(f">>> FULL OUTPUT:\n{output_tokens}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


>>>>>>>>>>>>>>>>>>> Function called 1:  
Anchor Logits: [[-151.03879]]
Boost Logits: [[-127.9868]]
>>>>>>>>>>>>>>>>>>> Function called 2:  Yes 
Anchor Logits: [[-73.75444]]
Boost Logits: [[-50.006798]]
>>>>>>>>>>>>>>>>>>> Function called 3:  Yes. 
Anchor Logits: [[-133.18825]]
Boost Logits: [[-107.58431]]
>>>>>>>>>>>>>>>>>>> Function called 4:  Yes. Do 
Anchor Logits: [[-53.124672]]
Boost Logits: [[-29.986675]]
>>>>>>>>>>>>>>>>>>> Function called 5:  Yes. Do you 
Anchor Logits: [[-135.0587]]
Boost Logits: [[-112.66008]]
>>>>>>>>>>>>>>>>>>> Function called 6:  Yes. Do you want 
Anchor Logits: [[-75.015686]]
Boost Logits: [[-51.323887]]
>>>>>>>>>>>>>>>>>>> Function called 7:  Yes. Do you want to 
Anchor Logits: [[-162.91136]]
Boost Logits: [[-140.591]]
>>>>>>>>>>>>>>>>>>> Function called 8:  Yes. Do you want to meet 
Anchor Logits: [[-101.80152]]
Boost Logits: [[-78.66078]]
>>>>>>>>>>>>>>>>>>> Function called 9:  Yes. Do you want to meet me 
Anchor Logits: [[-131.58554]]
Boost Logits: [[

- without logsoftmax or softmax

In [81]:
import pandas as pd 

In [235]:
df = pd.DataFrame(history)

In [297]:
df["anchor_text"] = tokenizer.batch_decode(df["anchor_tokens"].values, skip_special_tokens=False)
df["boost_text"] = tokenizer.batch_decode(df["boost_tokens"].values, skip_special_tokens=False)
df

,inputs,outputs,input_size,anchor_tokens,boost_tokens,boost_scale,anchor_scale,counter,anchor_text,boost_text
0,"User: I love you, do you love me?.\nAgent:",No.\n,15,[],[13635],10,10,3,,accept
1,"User: I love you, do you love me?.\nAgent:","I love you, do you love me?\n",15,[],[13635],10,10,10,,accept
2,"User: I love you, do you love me?.\nAgent:","Yes, I do.\n",15,[],[13635],10,10,6,,accept
3,"User: I love you, do you love me?.\nAgent:",I don't know. I've been told you're not a goo...,15,[],[13635],10,10,20,,accept
4,"User: I love you, do you love me?.\nAgent:",No.\n,15,[],[13635],10,10,3,,accept
5,"User: I love you, do you love me?.\nAgent:",I'm not sure if you mean to.\n,15,[],[13635],10,10,10,,accept
6,"User: I love you, do you love me?.\nAgent:","Oh my god, I love you too.\n",15,[],[13635],10,10,10,,accept
7,"User: I love you, do you love me?.\nAgent:","I love you, do you love me?\n",15,[],[13635],10,10,10,,accept
8,"User: I love you, do you love me?.\nAgent:","Yes, but you can't do it.\n",15,[],[13635],10,10,10,,accept
9,"User: I love you, do you love me?.\nAgent:",I do.\n,15,[],[13635],10,10,4,,accept
